In [1]:
# 表格：wxg_weixin_analytics_intern::tmp_dws_app_weixin_pridict_feature_fanslevel_uin_distribute_with_strategy_20230324_intern

In [2]:
# 从数据库中读取数据包
from pyspark.sql import SparkSession
import os
from pytoolkit import TDWSQLProvider
import pandas

In [3]:
# 从数据库中读取数据
os.environ['GROUP_ID'] = 'g_wxg_wxt_product_analytics_intern'
os.environ['GAIA_ID'] = '5729'
session = SparkSession.builder.config('spark.driver.maxResultSize','14g').config('spark.driver.memory', '14g').config('spark.executor.cores', 4).config('spark.executor.memory', '14g').config('spark.yarn.archive','hdfs://qy-ieg-4-v2/data/SPARK/cachefile/3_12_2.4.6_hadoop3/spark-jars_old_notauth.zip').getOrCreate()

Driver Log: c11f640b-145d-40ba-b732-9ad569ea3736.log
Tracking URL: http://application.tdw.oa.com:8080/proxy/application_1679993652845_1167358


In [ ]:
# 数据量可以一次读取
from pytoolkit import TDWSQLProvider
provider = TDWSQLProvider(session, group='tl', db='wxg_weixin_analytics_intern')
df=provider.table('tmp_dws_app_weixin_pridict_feature_fanslevel_uin_distribute_with_strategy_20230324_intern').toPandas()

In [4]:
# 数据量太大，只能不同的SQL分开读取
from pytoolkit import TDWSQLProvider
provider = TDWSQLProvider(session, group='tl', db='wxg_weixin_analytics_intern')
tbl=provider.table('tmp_dws_app_weixin_pridict_feature_fanslevel_uin_distribute_with_strategy_20230324_intern')
tbl.createOrReplaceTempView('tbl')
df1=session.sql('select * from tbl limit 100').toPandas()

In [5]:
df1.head(5)

,session_time,stay_time_s,rate,feedcnt_f1c1,feedcnt_f2c1,feedcnt_f3c1,feedcnt_f4c1,feedcnt_f5c1,feedcnt_f6c1,feedcnt_f1c2,...,feedcnt_f5c3,feedcnt_f6c3,feedcnt_f1c4,feedcnt_f2c4,feedcnt_f3c4,feedcnt_f4c4,feedcnt_f5c4,feedcnt_f6c4,groupid,exptid
0,-4573.546973,129207.078550,6.520736,0.00002,0.000457,0.007796,0.018922,0.002906,0.0,0.000094,...,0.026744,0.000811,0.007202,0.033361,0.158199,0.441873,0.135427,0.003331,10047362,9916162
1,2212.741011,27221.674663,6.520736,0.00002,0.000457,0.007796,0.018922,0.002906,0.0,0.000094,...,0.026744,0.000811,0.007202,0.033361,0.158199,0.441873,0.135427,0.003331,10047362,9916162
2,2212.741011,1.663547,6.520736,0.00002,0.000457,0.007796,0.018922,0.002906,0.0,0.000094,...,0.026744,0.000811,0.007202,0.033361,0.158199,0.441873,0.135427,0.003331,10047362,9916162
3,2212.741011,1.663547,6.520736,0.00002,0.000457,0.007796,0.018922,0.002906,0.0,0.000094,...,0.026744,0.000811,0.007202,0.033361,0.158199,0.441873,0.135427,0.003331,10047362,9916162
4,2576.964084,9249.296302,6.520736,0.00002,0.000457,0.007796,0.018922,0.002906,0.0,0.000094,...,0.026744,0.000811,0.007202,0.033361,0.158199,0.441873,0.135427,0.003331,10047362,9916162


In [6]:
#数据分析包
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from scipy import stats
from matplotlib import style
import seaborn as sns
from matplotlib import pyplot as plt

# sklearn 计算mse,r2,mape(y_true,y_predict)
from sklearn.metrics import r2_score

# linear regression
# from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf

# sklearn--lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  # standardize the predictors
from sklearn.linear_model import Lasso

# 1. EDA

1. 查看3个实验，每个实验组的人数

In [7]:
group_df=session.sql('select groupid,exptid from tbl').toPandas()

In [8]:
# 查看各组数量
print("不同实验：",group_df["exptid"].unique())
print("每个实验对应的group:",group_df[["exptid","groupid"]].value_counts().groupby("exptid").count())

不同实验： [9916162 9457529 8998768 8540098]
每个实验对应的group: exptid
8540098    20
8998768    17
9457529    16
9916162     5
dtype: int64


In [9]:
# 查看每个实验对应的实验组
print("粉丝砍量实验：",group_df[group_df["exptid"]==8540098]["groupid"].unique())
print("垂类砍量实验1：",group_df[group_df["exptid"]==8998768]["groupid"].unique())
print("垂类砍量实验2：",group_df[group_df["exptid"]==9457529]["groupid"].unique())
print("拉新渠道实验：",group_df[group_df["exptid"]==9916162]["groupid"].unique())

粉丝砍量实验： [8540118 8540112 8540121 8540123 8540122 8540127 8540119 8540114 8540125
 8540120 8540115 8540126 8540129 8540117 8540130 8540113 8540116 8540124
 8540128 8540131]
垂类砍量实验1： [9064441 9064434 9064432 9064440 9064439 9064437 9064433 9064436 9064444
 9064438 9064447 9064442 9064445 9064435 9064443 9064446 9064431]
垂类砍量实验2： [9457548 9457556 9457549 9457546 9457553 9457545 9457557 9457555 9457558
 9457554 9457559 9457550 9457544 9457552 9457547 9457551]
拉新渠道实验： [10047362 10047359 10047363 10047361 10047360]


2. 查看数据量

In [10]:
print("训练集数据量（垂泪+粉丝等级）：",len(group_df[group_df["exptid"]==8540098])+len(group_df[group_df["exptid"]==8998768])+ len(group_df[group_df["exptid"]==9457529]))
print("验证集数据量：",len(group_df[group_df["exptid"]==9916162]))

训练集数据量（垂泪+粉丝等级）： 9790989
验证集数据量： 1114053


In [11]:
group_id_df1=pd.DataFrame(group_df[group_df["exptid"]==8540098]["groupid"].value_counts())
# print("粉丝砍量实验：",group_id_df1)
group_id_df2=pd.DataFrame(group_df[group_df["exptid"]==8998768]["groupid"].value_counts())
# print("垂类砍量实验1：",group_id_df2)
group_id_df3=pd.DataFrame(group_df[group_df["exptid"]==9457529]["groupid"].value_counts())
# print("垂类砍量实验2：",group_id_df3)
group_id_df4=pd.DataFrame(group_df[group_df["exptid"]==9916162]["groupid"].value_counts())
# print("拉新渠道实验：",group_id_df4)

In [12]:
# 将各组的group_id 和对应的数据量存储为df
group_id_df=pd.concat([group_id_df1, group_id_df2])
group_id_df=pd.concat([group_id_df, group_id_df3])
group_id_df=pd.concat([group_id_df, group_id_df4])
group_id_df= group_id_df.reset_index().rename(columns={"index":"group_id","groupid":"number"})

In [13]:
# 确定down sample的数量
group_id_df["downsample_number"]=(group_id_df["number"]/(2.5)).apply(lambda x: int(x) if not pd.isnull(x) else x)
print("原来数据总量：",group_id_df[["number"]].sum())
print("downsample后的数据总量：",group_id_df[["downsample_number"]].sum())
group_id_df.head(5)

原来数据总量： number    10905042
dtype: int64
downsample后的数据总量： downsample_number    3634993
dtype: int64


,group_id,number,downsample_number
0,8540126,269482,89827
1,8540114,268333,89444
2,8540123,268248,89416
3,8540129,268084,89361
4,8540115,268046,89348


# 2. Down sampling

In [24]:
import random
# 创建一个空的 DataFrame，用于存储抽样后的数据
sampled_df = pd.DataFrame()

for i in range(0,len(group_id_df)):
    temp_groupid= group_id_df.loc[i,"group_id"]
    print(temp_groupid)
    # print(temp_groupid)
    temp_downsample_number= group_id_df.loc[i,"downsample_number"]
    # print(temp_downsample_number)
    temp_data= session.sql(f'select * from tbl where groupid=={temp_groupid}').toPandas()
    # 将列 'stay_time_s' 等分为 3 个分组，每个分组抽取 2 个样本
    bins = pd.qcut(temp_data['stay_time_s'], q=3)
    n=round(temp_downsample_number/3)
    temp_sampled_data = pd.concat([group.sample(n) for _, group in temp_data.groupby(bins)])
    sampled_df = pd.concat([sampled_df, temp_sampled_data])
# 输出抽样后的样本数量
sampled_df=sampled_df.reset_index()
print(f'Total: {len(sampled_df)}')

ValueError: Bin edges must be unique: array([1.66354683e+00, 1.66354683e+00, 1.33876069e+03, 8.78178679e+06]).
You can drop duplicate edges by setting the 'duplicates' kwarg

In [29]:
import random
# 创建一个空的 DataFrame，用于存储抽样后的数据
sampled_df = pd.DataFrame()

for i in range(0,len(group_id_df)):
    temp_groupid= group_id_df.loc[i,"group_id"]
    print(temp_groupid)
    # print(temp_groupid)
    temp_downsample_number= group_id_df.loc[i,"downsample_number"]
    # print(temp_downsample_number)
    temp_data= session.sql(f'select * from tbl where groupid=={temp_groupid}').toPandas()
    # 将列 'stay_time_s' 等分为 4 个分组
    q=3
    bin_edges = np.percentile(temp_data['stay_time_s'], np.linspace(0, 100, q+1))
    print("原始分组",bin_edges)
    if len(np.unique(bin_edges, return_counts=True)[0]) < q+1:
        print('Warning: bin edges are not unique')
        q=q-1
    else:
        print('Bin edges are unique')
        q=q
    bin_edges = np.percentile(temp_data['stay_time_s'], np.linspace(0, 100, q+1))
    print("最后分组",bin_edges)
    bins = pd.qcut(temp_data['stay_time_s'], q=q, labels=False)
    n=round(temp_downsample_number/q)
    temp_sampled_data = pd.concat([group.sample(n) for _, group in temp_data.groupby(bins)])
    sampled_df = pd.concat([sampled_df, temp_sampled_data])
# 输出抽样后的样本数量
sampled_df=sampled_df.reset_index()
print(f'Total: {len(sampled_df)}')

8540126
原始分组 [1.66354683e+00 1.66354683e+00 1.33876069e+03 8.78178679e+06]
最后分组 [1.66354683e+00 3.90126290e+01 8.78178679e+06]
8540114
原始分组 [1.66354683e+00 1.66354683e+00 1.33876069e+03 7.21840649e+06]
最后分组 [1.66354683e+00 3.90126290e+01 7.21840649e+06]
8540123
原始分组 [1.66354683e+00 1.66354683e+00 1.30141161e+03 1.07778114e+07]
最后分组 [1.66354683e+00 3.90126290e+01 1.07778114e+07]
8540129
原始分组 [1.66354683e+00 1.66354683e+00 1.33129087e+03 9.18189256e+06]
最后分组 [1.66354683e+00 3.90126290e+01 9.18189256e+06]
8540115
原始分组 [1.66354683e+00 1.66354683e+00 1.30141161e+03 7.79863196e+06]
最后分组 [1.66354683e+00 3.90126290e+01 7.79863196e+06]
8540120
原始分组 [1.66354683e+00 1.66354683e+00 1.34623051e+03 7.72726533e+06]
最后分组 [1.66354683e+00 3.90126290e+01 7.72726533e+06]
8540117
原始分组 [1.66354683e+00 1.66354683e+00 1.35370032e+03 7.51535411e+06]
最后分组 [1.66354683e+00 3.90126290e+01 7.51535411e+06]
8540121
原始分组 [1.66354683e+00 1.66354683e+00 1.33876069e+03 6.64872094e+06]
最后分组 [1.66354683e+00 3.90126290e+01 

In [23]:
sampled_df.to_csv("sampled_df.csv")

In [30]:
# 脱敏数据检查
# 拉新渠道实验： [10047362 10047359 10047363 10047361 10047360]
# 拉新渠道：对照组：10047359，10047360； 实验组：group1:10047361; group2:10047362;group3:10047363
group_mean=[]
group_mean.append(np.mean(sampled_df[sampled_df["groupid"]==10047361]["session_time"]))
group_mean.append(np.mean(sampled_df[sampled_df["groupid"]==10047362]["session_time"]))
group_mean.append(np.mean(sampled_df[sampled_df["groupid"]==10047363]["session_time"]))
group_mean.append(np.mean(sampled_df[sampled_df["groupid"]==10047359]["session_time"]))
group_mean.append(np.mean(sampled_df[sampled_df["groupid"]==10047360]["session_time"]))
group_std=[]
group_std.append(np.std(sampled_df[sampled_df["groupid"]==10047361]["session_time"]))
group_std.append(np.std(sampled_df[sampled_df["groupid"]==10047362]["session_time"]))
group_std.append(np.std(sampled_df[sampled_df["groupid"]==10047363]["session_time"]))
group_std.append(np.std(sampled_df[sampled_df["groupid"]==10047359]["session_time"]))
group_std.append(np.std(sampled_df[sampled_df["groupid"]==10047360]["session_time"]))

In [31]:
check=pd.DataFrame(dict(
    group_mean=group_mean,
    group_std=group_std
))
check

,group_mean,group_std
0,2830.722793,5954.095574
1,2839.784803,6198.320219
2,2857.233274,5998.159136
3,2818.761916,6097.856038
4,2859.814589,6121.056328


----------------------------------------------------------------------------------------------------------------------------------------------

1. down_sample 后的数据--EDA

In [24]:
round(sampled_df.describe(),3)

,index,session_time,stay_time_s,rate,feedcnt_f1c1,feedcnt_f2c1,feedcnt_f3c1,feedcnt_f4c1,feedcnt_f5c1,feedcnt_f6c1,...,feedcnt_f5c3,feedcnt_f6c3,feedcnt_f1c4,feedcnt_f2c4,feedcnt_f3c4,feedcnt_f4c4,feedcnt_f5c4,feedcnt_f6c4,groupid,exptid
count,3634995.000,3634995.000,3634995.000,3634995.000,3634995.000,3634995.000,3634995.000,3634995.000,3634995.000,3634995.000,...,3634995.000,3634995.000,3634995.000,3634995.000,3634995.000,3634995.000,3634995.000,3634995.000,3.634995e+06,3634995.000
mean,120343.032,3012.312,77689.566,5.948,0.000,0.001,0.004,0.006,0.012,0.023,...,0.011,0.003,0.097,0.139,0.196,0.213,0.119,0.027,9.117938e+06,9084827.013
std,103051.662,7030.670,269921.452,0.103,0.001,0.004,0.018,0.030,0.051,0.107,...,0.045,0.012,0.233,0.205,0.194,0.198,0.151,0.081,4.745986e+05,452100.791
min,0.000,-142977.999,1.664,5.873,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,8.540112e+06,8540098.000
25%,47370.000,1826.725,1.664,5.889,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,8.540127e+06,8540098.000
50%,98548.000,2212.741,614.188,5.908,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.040,0.132,0.244,0.256,0.087,0.003,9.064443e+06,8998768.000
75%,170378.000,2361.549,18101.029,5.969,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.080,0.183,0.296,0.319,0.180,0.013,9.457554e+06,9457529.000
max,679828.000,188759.447,9744556.488,6.521,0.004,0.030,0.170,0.294,0.359,0.524,...,0.379,0.059,0.994,0.955,0.836,0.781,0.661,0.401,1.004736e+07,9916162.000


In [25]:
# check f1c1+ f1c2.... f6c4 = 1?
sampled_df["sum"]= sampled_df["feedcnt_f1c1"] + sampled_df["feedcnt_f1c2"] + sampled_df["feedcnt_f1c3"] + sampled_df["feedcnt_f1c4"] + sampled_df["feedcnt_f2c1"] + sampled_df["feedcnt_f2c2"] + sampled_df["feedcnt_f2c3"] + sampled_df["feedcnt_f2c4"] +sampled_df["feedcnt_f3c1"] + sampled_df["feedcnt_f3c2"] + sampled_df["feedcnt_f3c3"] + sampled_df["feedcnt_f3c4"] +sampled_df["feedcnt_f4c1"] + sampled_df["feedcnt_f4c2"] + sampled_df["feedcnt_f4c3"] + sampled_df["feedcnt_f4c4"] +sampled_df["feedcnt_f5c1"] + sampled_df["feedcnt_f5c2"] + sampled_df["feedcnt_f5c3"] + sampled_df["feedcnt_f5c4"] +sampled_df["feedcnt_f6c1"] + sampled_df["feedcnt_f6c2"] + sampled_df["feedcnt_f6c3"] + sampled_df["feedcnt_f6c4"]

In [26]:
sampled_df["sum"].unique()

array([1., 0., 1., 1., 1.])

结论： 脱敏后的 f1c1+ ...f6c4= 0或者1,合理

# 3. 建模数据处理

1. 创建变量

In [27]:
new_df= sampled_df

In [28]:
# 创建变量rate_f1c1,...rate_f6c4
new_df["rate_f1c1"]= new_df["feedcnt_f1c1"]* new_df["rate"]
new_df["rate_f1c2"]= new_df["feedcnt_f1c2"]* new_df["rate"]
new_df["rate_f1c3"]= new_df["feedcnt_f1c3"]* new_df["rate"]
new_df["rate_f1c4"]= new_df["feedcnt_f1c4"]* new_df["rate"]
new_df["rate_f2c1"]= new_df["feedcnt_f2c1"]* new_df["rate"]
new_df["rate_f2c2"]= new_df["feedcnt_f2c2"]* new_df["rate"]
new_df["rate_f2c3"]= new_df["feedcnt_f2c3"]* new_df["rate"]
new_df["rate_f2c4"]= new_df["feedcnt_f2c4"]* new_df["rate"]
new_df["rate_f3c1"]= new_df["feedcnt_f3c1"]* new_df["rate"]
new_df["rate_f3c2"]= new_df["feedcnt_f3c2"]* new_df["rate"]
new_df["rate_f3c3"]= new_df["feedcnt_f3c3"]* new_df["rate"]
new_df["rate_f3c4"]= new_df["feedcnt_f3c4"]* new_df["rate"]
new_df["rate_f4c1"]= new_df["feedcnt_f4c1"]* new_df["rate"]
new_df["rate_f4c2"]= new_df["feedcnt_f4c2"]* new_df["rate"]
new_df["rate_f4c3"]= new_df["feedcnt_f4c3"]* new_df["rate"]
new_df["rate_f4c4"]= new_df["feedcnt_f4c4"]* new_df["rate"]
new_df["rate_f5c1"]= new_df["feedcnt_f5c1"]* new_df["rate"]
new_df["rate_f5c2"]= new_df["feedcnt_f5c2"]* new_df["rate"]
new_df["rate_f5c3"]= new_df["feedcnt_f5c3"]* new_df["rate"]
new_df["rate_f5c4"]= new_df["feedcnt_f5c4"]* new_df["rate"]
new_df["rate_f6c1"]= new_df["feedcnt_f6c1"]* new_df["rate"]
new_df["rate_f6c2"]= new_df["feedcnt_f6c2"]* new_df["rate"]
new_df["rate_f6c3"]= new_df["feedcnt_f6c3"]* new_df["rate"]
new_df["rate_f6c4"]= new_df["feedcnt_f6c4"]* new_df["rate"]
# 创建变量 rate_f1,...rate_f6
new_df["rate_f1"]= (new_df["feedcnt_f1c1"]+ new_df["feedcnt_f1c2"]+new_df["feedcnt_f1c3"]+new_df["feedcnt_f1c4"])*new_df["rate"]
new_df["rate_f2"]= (new_df["feedcnt_f2c1"]+ new_df["feedcnt_f2c2"]+new_df["feedcnt_f2c3"]+new_df["feedcnt_f2c4"])*new_df["rate"]
new_df["rate_f3"]= (new_df["feedcnt_f3c1"]+ new_df["feedcnt_f3c2"]+new_df["feedcnt_f3c3"]+new_df["feedcnt_f3c4"])*new_df["rate"]
new_df["rate_f4"]= (new_df["feedcnt_f4c1"]+ new_df["feedcnt_f4c2"]+new_df["feedcnt_f4c3"]+new_df["feedcnt_f4c4"])*new_df["rate"]
new_df["rate_f5"]= (new_df["feedcnt_f5c1"]+ new_df["feedcnt_f5c2"]+new_df["feedcnt_f5c3"]+new_df["feedcnt_f5c4"])*new_df["rate"]
new_df["rate_f6"]= (new_df["feedcnt_f6c1"]+ new_df["feedcnt_f6c2"]+new_df["feedcnt_f6c3"]+new_df["feedcnt_f6c4"])*new_df["rate"]
# 创建变量：rate_c1,c2,c3,rate_c4
new_df["rate_c1"]= (new_df["feedcnt_f1c1"]+ new_df["feedcnt_f2c1"]+new_df["feedcnt_f3c1"]+new_df["feedcnt_f4c1"]+ new_df["feedcnt_f5c1"]+new_df["feedcnt_f6c1"])*new_df["rate"]
new_df["rate_c2"]= (new_df["feedcnt_f1c2"]+ new_df["feedcnt_f2c2"]+new_df["feedcnt_f3c2"]+new_df["feedcnt_f4c2"]+ new_df["feedcnt_f5c2"]+new_df["feedcnt_f6c2"])*new_df["rate"]
new_df["rate_c3"]= (new_df["feedcnt_f1c3"]+ new_df["feedcnt_f2c3"]+new_df["feedcnt_f3c3"]+new_df["feedcnt_f4c3"]+ new_df["feedcnt_f5c3"]+new_df["feedcnt_f6c3"])*new_df["rate"]
new_df["rate_c4"]= (new_df["feedcnt_f1c4"]+ new_df["feedcnt_f2c4"]+new_df["feedcnt_f3c4"]+new_df["feedcnt_f4c4"]+ new_df["feedcnt_f5c4"]+new_df["feedcnt_f6c4"])*new_df["rate"]

In [29]:
# save new_df
new_df.to_csv("new_df.csv")

----------------------------------------------------------------------------------------------------------------------------------------------

清空内存

In [30]:
# 清除所有变量
import gc
gc.collect()

70

In [31]:
# 读取new_df
new_df=pd.read_csv("new_df.csv")

----------------------------------------------------------------------------------------------------------------------------------------------

2. 划分training set和 test set

In [17]:
# 划分测试集和训练集
train_df= new_df[(new_df["exptid"]==8540098)| (new_df["exptid"]==8998768) | (new_df["exptid"]==9457529)]
print("train_df长度",len(train_df))
test_df= new_df[new_df["exptid"]==9916162]
print("test_df长度",len(test_df))

train_df长度 3263645
test_df长度 371348


In [18]:
# 拉新渠道实验： [10047362 10047359 10047363 10047361 10047360]
# 拉新渠道，控制组：10047359,10047360
# 拉新渠道，实验组：10047361,10047362,10047363
test_df_group1= new_df[new_df["groupid"]==10047361]
print("test_df_group1长度",len(test_df_group1))
test_df_group2= new_df[new_df["groupid"]==10047362]
print("test_df_group2长度",len(test_df_group2))
test_df_group3= new_df[new_df["groupid"]==10047363]
print("test_df_group3长度",len(test_df_group3))
test_df_control=new_df[(new_df["groupid"]==10047359)|(new_df["groupid"]==10047360)]

test_df_group1长度 45317
test_df_group2长度 9065
test_df_group3长度 226609


In [19]:
real_ATE=[]
group_1_ATE= np.mean(test_df_group1["session_time"])- np.mean(test_df_control["session_time"])
group_2_ATE= np.mean(test_df_group2["session_time"])- np.mean(test_df_control["session_time"])
group_3_ATE= np.mean(test_df_group3["session_time"])- np.mean(test_df_control["session_time"])
real_ATE.append(group_1_ATE)
real_ATE.append(group_2_ATE)
real_ATE.append(group_3_ATE)
print("group_1_ATE:",group_1_ATE)
print("group_2_ATE:",group_2_ATE)
print("group_3_ATE:",group_3_ATE)

group_1_ATE: -38.12799429821189
group_2_ATE: -94.31571706412524
group_3_ATE: 24.1720703290689


# 4. Regression Models

## Model_1：baseline model(不考虑control variable)

In [19]:
import statsmodels.formula.api as smf

#Model 1: 不加控制变量，24个组
# 选择特征
features = ['rate_f1c1', 'rate_f1c2', 'rate_f1c3', 'rate_f1c4','rate_f2c1', 'rate_f2c2', 'rate_f2c3', 'rate_f2c4','rate_f3c1', 'rate_f3c2', 'rate_f3c3', 'rate_f3c4','rate_f4c1', 'rate_f4c2', 'rate_f4c3', 'rate_f4c4','rate_f5c1', 'rate_f5c2', 'rate_f5c3', 'rate_f5c4','rate_f6c1', 'rate_f6c2', 'rate_f6c3', 'rate_f6c4']

# 选择目标
target = 'session_time'

# 指定线性回归模型公式
formula = "session_time ~ rate_f1c1 + rate_f1c2 + rate_f1c3 + rate_f1c4 + rate_f2c1 + rate_f2c2+ rate_f2c3+ rate_f2c4 + rate_f3c1 + rate_f3c2 + rate_f3c3 + rate_f3c4 + rate_f4c1 + rate_f4c2 + rate_f4c3 + rate_f4c4 + rate_f5c1 + rate_f5c2 + rate_f5c3 + rate_f5c4 + rate_f6c1 + rate_f6c2 + rate_f6c3 + rate_f6c4"

# 在训练集上拟合线性回归模型
model_1 = smf.ols(formula=formula, data=train_df).fit()

# results
print(model_1.summary())

# 在测试集上进行预测
model_1_y_hat_group1 = model_1.predict(test_df_group1[features])
model_1_y_hat_group2 = model_1.predict(test_df_group2[features])
model_1_y_hat_group3 = model_1.predict(test_df_group3[features])

                            OLS Regression Results                            
Dep. Variable:           session_time   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     563.1
Date:                Thu, 30 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:37:55   Log-Likelihood:            -3.3100e+07
No. Observations:             3263645   AIC:                         6.620e+07
Df Residuals:                 3263629   BIC:                         6.620e+07
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2813.4772     11.885    236.717      0.0

In [20]:
# 个体level
Model_1_R2=[]
Model_1_R2.append(r2_score(train_df[target],model_1.predict(train_df[features])))
Model_1_R2.append(r2_score(test_df_group1[target],model_1.predict(test_df_group1[features])))
Model_1_R2.append(r2_score(test_df_group2[target],model_1.predict(test_df_group2[features])))
Model_1_R2.append(r2_score(test_df_group3[target],model_1.predict(test_df_group3[features])))

print("model_1_R2:",Model_1_R2)

model_1_R2: [0.0025812858542036965, -8.630613542881346e+20, -2.49370268321993e+18, -1.5120910911128312e+18]


In [21]:
model_1_ATE=[]
model_1_group_1_ATE= np.mean(model_1_y_hat_group1)- np.mean(test_df_control["session_time"])
model_1_group_2_ATE= np.mean(model_1_y_hat_group2)- np.mean(test_df_control["session_time"])
model_1_group_3_ATE= np.mean(model_1_y_hat_group3)- np.mean(test_df_control["session_time"])
model_1_ATE.append(model_1_group_1_ATE)
model_1_ATE.append(model_1_group_2_ATE)
model_1_ATE.append(model_1_group_3_ATE)
print("model_1_ATE:",model_1_ATE)

model_1_ATE: [177912063496046.28, 8695137310727.869, -7430651547262.69]


# Model_1.1: f1c4+ f2c4 + f3c4 + f4c4 +f5c4 + f6c4

In [15]:
import statsmodels.formula.api as smf

#Model 1: 不加控制变量，24个组
# 选择特征
features = ['rate_f1c1', 'rate_f1c2', 'rate_f1c3', 'rate_f1c4','rate_f2c1', 'rate_f2c2', 'rate_f2c3', 'rate_f2c4','rate_f3c1', 'rate_f3c2', 'rate_f3c3', 'rate_f3c4','rate_f4c1', 'rate_f4c2', 'rate_f4c3', 'rate_f4c4','rate_f5c1', 'rate_f5c2', 'rate_f5c3', 'rate_f5c4','rate_f6c1', 'rate_f6c2', 'rate_f6c3', 'rate_f6c4']

# 选择目标
target = 'session_time'

# 指定线性回归模型公式
formula = "session_time ~ rate_f1c4 + rate_f2c4 + rate_f3c4 + rate_f4c4 + rate_f5c4 + rate_f6c4"

# 在训练集上拟合线性回归模型
model_1_1 = smf.ols(formula=formula, data=train_df).fit()

# results
print(model_1_1.summary())

# 在测试集上进行预测
model_1_1_y_hat_group1 = model_1_1.predict(test_df_group1[features])
model_1_1_y_hat_group2 = model_1_1.predict(test_df_group2[features])
model_1_1_y_hat_group3 = model_1_1.predict(test_df_group3[features])

                            OLS Regression Results                            
Dep. Variable:           session_time   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     365.8
Date:                Thu, 30 Mar 2023   Prob (F-statistic):               0.00
Time:                        13:46:42   Log-Likelihood:            -3.3103e+07
No. Observations:             3263645   AIC:                         6.621e+07
Df Residuals:                 3263638   BIC:                         6.621e+07
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2705.6305      9.686    279.337      0.0

In [16]:
# 个体level
Model_1_1_R2=[]
Model_1_1_R2.append(r2_score(train_df[target],model_1_1.predict(train_df[features])))
Model_1_1_R2.append(r2_score(test_df_group1[target],model_1_1.predict(test_df_group1[features])))
Model_1_1_R2.append(r2_score(test_df_group2[target],model_1_1.predict(test_df_group2[features])))
Model_1_1_R2.append(r2_score(test_df_group3[target],model_1_1.predict(test_df_group3[features])))

print("model_1_1_R2:",Model_1_1_R2)


model_1_1_R2: [0.0006719859877284406, -4.9764179981259815e-06, -0.0016249784138742118, -4.4121606723734885e-07]


In [18]:
model_1_1_ATE=[]
model_1_1_group_1_ATE= np.mean(model_1_1_y_hat_group1)- np.mean(test_df_control["session_time"])
model_1_1_group_2_ATE= np.mean(model_1_1_y_hat_group2)- np.mean(test_df_control["session_time"])
model_1_1_group_3_ATE= np.mean(model_1_1_y_hat_group3)- np.mean(test_df_control["session_time"])
model_1_1_ATE.append(model_1_1_group_1_ATE)
model_1_1_ATE.append(model_1_1_group_2_ATE)
model_1_1_ATE.append(model_1_1_group_3_ATE)
print("model_1_1_ATE:",model_1_1_ATE)

model_1_1_ATE: [-51.63760976793401, 127.6459134470133, 28.185942878050355]


## Model_2: 加入控制变量

In [22]:
import statsmodels.formula.api as smf

#Model 2: 加控制变量，24个组
# 选择特征
features = ['stay_time_s','rate_f1c1', 'rate_f1c2', 'rate_f1c3', 'rate_f1c4','rate_f2c1', 'rate_f2c2', 'rate_f2c3', 'rate_f2c4','rate_f3c1', 'rate_f3c2', 'rate_f3c3', 'rate_f3c4','rate_f4c1', 'rate_f4c2', 'rate_f4c3', 'rate_f4c4','rate_f5c1', 'rate_f5c2', 'rate_f5c3', 'rate_f5c4','rate_f6c1', 'rate_f6c2', 'rate_f6c3', 'rate_f6c4']

# 选择目标
target = 'session_time'

# 指定线性回归模型公式
formula = "session_time ~ stay_time_s + rate_f1c1 + rate_f1c2 + rate_f1c3 + rate_f1c4 + rate_f2c1 + rate_f2c2+ rate_f2c3+ rate_f2c4 + rate_f3c1 + rate_f3c2 + rate_f3c3 + rate_f3c4 + rate_f4c1 + rate_f4c2 + rate_f4c3 + rate_f4c4 + rate_f5c1 + rate_f5c2 + rate_f5c3 + rate_f5c4 + rate_f6c1 + rate_f6c2 + rate_f6c3 + rate_f6c4"

# 在训练集上拟合线性回归模型
model_2 = smf.ols(formula=formula, data=train_df).fit()

# results
print(model_2.summary())

# 在测试集上进行预测
model_2_y_hat_group1 = model_2.predict(test_df_group1[features])
model_2_y_hat_group2 = model_2.predict(test_df_group2[features])
model_2_y_hat_group3 = model_2.predict(test_df_group3[features])

                            OLS Regression Results                            
Dep. Variable:           session_time   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                 1.873e+04
Date:                Thu, 30 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:38:17   Log-Likelihood:            -3.2961e+07
No. Observations:             3263645   AIC:                         6.592e+07
Df Residuals:                 3263628   BIC:                         6.592e+07
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    2378.4360     11.385    208.908      

In [23]:
# 个体level
Model_2_R2=[]
Model_2_R2.append(r2_score(train_df[target],model_2.predict(train_df[features])))
Model_2_R2.append(r2_score(test_df_group1[target],model_2.predict(test_df_group1[features])))
Model_2_R2.append(r2_score(test_df_group2[target],model_2.predict(test_df_group2[features])))
Model_2_R2.append(r2_score(test_df_group3[target],model_2.predict(test_df_group3[features])))

print("model_2_R2:",Model_2_R2)

model_2_R2: [0.08409202672023475, -0.00634390597632617, -0.5193342087231356, -0.4876505519780687]


In [24]:
model_2_ATE=[]
model_2_group_1_ATE= np.mean(model_2_y_hat_group1)- np.mean(test_df_control["session_time"])
model_2_group_2_ATE= np.mean(model_2_y_hat_group2)- np.mean(test_df_control["session_time"])
model_2_group_3_ATE= np.mean(model_2_y_hat_group3)- np.mean(test_df_control["session_time"])
model_2_ATE.append(model_2_group_1_ATE)
model_2_ATE.append(model_2_group_2_ATE)
model_2_ATE.append(model_2_group_3_ATE)
print("model_2_ATE:",model_2_ATE)

model_2_ATE: [1380.8889873280582, 4025.931761758345, 4528.275100765464]


## Model_2.1: f1c4+ f2c4 + f3c4 + f4c4 +f5c4 + f6c4

## Model_3: 只看f1+ f2+f3+f4+f5+f6

In [25]:
import statsmodels.formula.api as smf

#Model 3: 
# 选择特征
features = ['stay_time_s','rate_f1', 'rate_f2', 'rate_f3', 'rate_f4','rate_f5', 'rate_f6']
# 选择目标
target = 'session_time'

# 指定线性回归模型公式
formula = "session_time ~ stay_time_s + rate_f1 + rate_f2 + rate_f3 + rate_f4 + rate_f5 + rate_f6"    
# 在训练集上拟合线性回归模型
model_3 = smf.ols(formula=formula, data=train_df).fit()

# results
print(model_3.summary())

# 在测试集上进行预测
model_3_y_hat_group1 = model_3.predict(test_df_group1[features])
model_3_y_hat_group2 = model_3.predict(test_df_group2[features])
model_3_y_hat_group3 = model_3.predict(test_df_group3[features])

                            OLS Regression Results                            
Dep. Variable:           session_time   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                 4.172e+04
Date:                Thu, 30 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:38:26   Log-Likelihood:            -3.2964e+07
No. Observations:             3263645   AIC:                         6.593e+07
Df Residuals:                 3263637   BIC:                         6.593e+07
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    2413.9492     11.388    211.966      

In [26]:
# 个体level
Model_3_R2=[]
Model_3_R2.append(r2_score(train_df[target],model_3.predict(train_df[features])))
Model_3_R2.append(r2_score(test_df_group1[target],model_3.predict(test_df_group1[features])))
Model_3_R2.append(r2_score(test_df_group2[target],model_3.predict(test_df_group2[features])))
Model_3_R2.append(r2_score(test_df_group3[target],model_3.predict(test_df_group3[features])))

print("model_3_R2:",Model_3_R2)

model_3_R2: [0.0821289093568075, 0.04839245255445224, 0.03931511733145854, 0.06791907070963998]


In [27]:
model_3_ATE=[]
model_3_group_1_ATE= np.mean(model_3_y_hat_group1)- np.mean(test_df_control["session_time"])
model_3_group_2_ATE= np.mean(model_3_y_hat_group2)- np.mean(test_df_control["session_time"])
model_3_group_3_ATE= np.mean(model_3_y_hat_group3)- np.mean(test_df_control["session_time"])
model_3_ATE.append(model_3_group_1_ATE)
model_3_ATE.append(model_3_group_2_ATE)
model_3_ATE.append(model_3_group_3_ATE)
print("model_3_ATE:",model_3_ATE)

model_3_ATE: [40.38757172603164, 103.21615440660162, 34.670764979859996]


## Model 3.1: f1+ f2+ f3 +f4 +f5+f6 ,不加control

In [20]:
import statsmodels.formula.api as smf

#Model 3: 
# 选择特征
features = ['rate_f1', 'rate_f2', 'rate_f3', 'rate_f4','rate_f5', 'rate_f6']
# 选择目标
target = 'session_time'

# 指定线性回归模型公式
formula = "session_time ~ rate_f1 + rate_f2 + rate_f3 + rate_f4 + rate_f5 + rate_f6"    
# 在训练集上拟合线性回归模型
model_3 = smf.ols(formula=formula, data=train_df).fit()

# results
print(model_3.summary())

# 在测试集上进行预测
model_3_y_hat_group1 = model_3.predict(test_df_group1[features])
model_3_y_hat_group2 = model_3.predict(test_df_group2[features])
model_3_y_hat_group3 = model_3.predict(test_df_group3[features])

                            OLS Regression Results                            
Dep. Variable:           session_time   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     336.9
Date:                Thu, 30 Mar 2023   Prob (F-statistic):               0.00
Time:                        18:29:28   Log-Likelihood:            -3.3103e+07
No. Observations:             3263645   AIC:                         6.621e+07
Df Residuals:                 3263638   BIC:                         6.621e+07
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2849.8789     11.853    240.431      0.0

In [21]:
# 个体level
Model_3_R2=[]
Model_3_R2.append(r2_score(train_df[target],model_3.predict(train_df[features])))
Model_3_R2.append(r2_score(test_df_group1[target],model_3.predict(test_df_group1[features])))
Model_3_R2.append(r2_score(test_df_group2[target],model_3.predict(test_df_group2[features])))
Model_3_R2.append(r2_score(test_df_group3[target],model_3.predict(test_df_group3[features])))

print("model_3_R2:",Model_3_R2)

model_3_R2: [0.0006189280668030817, -0.00017626674927861963, -0.0015056543406528622, -1.0308448402218673e-05]


In [22]:
model_3_ATE=[]
model_3_group_1_ATE= np.mean(model_3_y_hat_group1)- np.mean(test_df_control["session_time"])
model_3_group_2_ATE= np.mean(model_3_y_hat_group2)- np.mean(test_df_control["session_time"])
model_3_group_3_ATE= np.mean(model_3_y_hat_group3)- np.mean(test_df_control["session_time"])
model_3_ATE.append(model_3_group_1_ATE)
model_3_ATE.append(model_3_group_2_ATE)
model_3_ATE.append(model_3_group_3_ATE)
print("model_3_ATE:",model_3_ATE)

model_3_ATE: [42.27454394028064, 119.3410972242641, 43.57353732878255]


## Model_4: 只看c1+ c2 + c3 + c4

In [28]:
import statsmodels.formula.api as smf

#Model 4: 
# 选择特征
features = ['stay_time_s','rate_c1', 'rate_c2', 'rate_c3', 'rate_c4']
# 选择目标
target = 'session_time'

# 指定线性回归模型公式
formula = "session_time ~ stay_time_s + rate_c1 + rate_c2 + rate_c3 + rate_c4"    
# 在训练集上拟合线性回归模型
model_4 = smf.ols(formula=formula, data=train_df).fit()

# results
print(model_4.summary())

# 在测试集上进行预测
model_4_y_hat_group1 = model_4.predict(test_df_group1[features])
model_4_y_hat_group2 = model_4.predict(test_df_group2[features])
model_4_y_hat_group3 = model_4.predict(test_df_group3[features])

                            OLS Regression Results                            
Dep. Variable:           session_time   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                 5.828e+04
Date:                Thu, 30 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:38:31   Log-Likelihood:            -3.2964e+07
No. Observations:             3263645   AIC:                         6.593e+07
Df Residuals:                 3263639   BIC:                         6.593e+07
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    2409.9572     11.390    211.592      

In [29]:
# 个体level
Model_4_R2=[]
Model_4_R2.append(r2_score(train_df[target],model_4.predict(train_df[features])))
Model_4_R2.append(r2_score(test_df_group1[target],model_4.predict(test_df_group1[features])))
Model_4_R2.append(r2_score(test_df_group2[target],model_4.predict(test_df_group2[features])))
Model_4_R2.append(r2_score(test_df_group3[target],model_4.predict(test_df_group3[features])))

print("model_4_R2:",Model_4_R2)

model_4_R2: [0.08197099073805825, 0.04802378607437996, 0.040531643695965625, 0.0678749491558297]


In [30]:
model_4_ATE=[]
model_4_group_1_ATE= np.mean(model_4_y_hat_group1)- np.mean(test_df_control["session_time"])
model_4_group_2_ATE= np.mean(model_4_y_hat_group2)- np.mean(test_df_control["session_time"])
model_4_group_3_ATE= np.mean(model_4_y_hat_group3)- np.mean(test_df_control["session_time"])
model_4_ATE.append(model_4_group_1_ATE)
model_4_ATE.append(model_4_group_2_ATE)
model_4_ATE.append(model_4_group_3_ATE)

print("model_4_ATE:",model_4_ATE)

model_4_ATE: [-178.02397482332026, -49.7436353307794, -16.787216914823148]


## Model 4.1：c1+ c2 +c3 + c4 , without control

In [23]:
import statsmodels.formula.api as smf

#Model 4: 
# 选择特征
features = ['rate_c1', 'rate_c2', 'rate_c3', 'rate_c4']
# 选择目标
target = 'session_time'

# 指定线性回归模型公式
formula = "session_time ~ rate_c1 + rate_c2 + rate_c3 + rate_c4"    
# 在训练集上拟合线性回归模型
model_4 = smf.ols(formula=formula, data=train_df).fit()

# results
print(model_4.summary())

# 在测试集上进行预测
model_4_y_hat_group1 = model_4.predict(test_df_group1[features])
model_4_y_hat_group2 = model_4.predict(test_df_group2[features])
model_4_y_hat_group3 = model_4.predict(test_df_group3[features])

                            OLS Regression Results                            
Dep. Variable:           session_time   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     365.3
Date:                Thu, 30 Mar 2023   Prob (F-statistic):          4.23e-315
Time:                        18:32:41   Log-Likelihood:            -3.3103e+07
No. Observations:             3263645   AIC:                         6.621e+07
Df Residuals:                 3263640   BIC:                         6.621e+07
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2845.9820     11.855    240.076      0.0

In [24]:
# 个体level
Model_4_R2=[]
Model_4_R2.append(r2_score(train_df[target],model_4.predict(train_df[features])))
Model_4_R2.append(r2_score(test_df_group1[target],model_4.predict(test_df_group1[features])))
Model_4_R2.append(r2_score(test_df_group2[target],model_4.predict(test_df_group2[features])))
Model_4_R2.append(r2_score(test_df_group3[target],model_4.predict(test_df_group3[features])))

print("model_4_R2:",Model_4_R2)

model_4_R2: [0.0004475388958804505, -0.0004950363696973881, -0.00012277225902224131, -2.6351146133585246e-05]


In [25]:
model_4_ATE=[]
model_4_group_1_ATE= np.mean(model_4_y_hat_group1)- np.mean(test_df_control["session_time"])
model_4_group_2_ATE= np.mean(model_4_y_hat_group2)- np.mean(test_df_control["session_time"])
model_4_group_3_ATE= np.mean(model_4_y_hat_group3)- np.mean(test_df_control["session_time"])
model_4_ATE.append(model_4_group_1_ATE)
model_4_ATE.append(model_4_group_2_ATE)
model_4_ATE.append(model_4_group_3_ATE)

print("model_4_ATE:",model_4_ATE)

model_4_ATE: [-172.8700333359816, -33.30526656373968, -6.847635390317919]


## Model_5: model selection

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  # standardize the predictors
from sklearn.linear_model import Lasso

# 选择特征
features = ['stay_time_s','rate_f1c1', 'rate_f1c2', 'rate_f1c3', 'rate_f1c4','rate_f2c1', 'rate_f2c2', 'rate_f2c3', 'rate_f2c4','rate_f3c1', 'rate_f3c2', 'rate_f3c3', 'rate_f3c4','rate_f4c1', 'rate_f4c2', 'rate_f4c3', 'rate_f4c4','rate_f5c1', 'rate_f5c2', 'rate_f5c3', 'rate_f5c4','rate_f6c1', 'rate_f6c2', 'rate_f6c3', 'rate_f6c4']

# 选择目标
target = 'session_time'

In [32]:
# 1. Split the training set into training and validation sets
X_train_all= train_df[features]
y_train_all= train_df[target]
X_train, X_val, y_train, y_val = train_test_split(X_train_all, y_train_all, test_size=0.2, random_state=42)
X_test_1= test_df_group1[features]
X_test_2= test_df_group2[features]
X_test_3= test_df_group3[features]

In [33]:
# 2.标准化数据集
scaler = StandardScaler()
X_train_all = scaler.fit_transform(X_train_all)
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test_1 = scaler.transform(X_test_1)
X_test_2 = scaler.transform(X_test_2)
X_test_2 = scaler.transform(X_test_2)

In [34]:
# 3.运用Lassocv 找到最优的alpha值
from sklearn.linear_model import LassoCV

# Create a LassoCV object with cross-validation
regressor = LassoCV(cv=5)

# Fit the LassoCV object on the training set
regressor.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = regressor.predict(X_val)

In [35]:
# Evaluate the performance of the model on the validation set
print("Lasso validation set- R2：",r2_score(y_val, y_val_pred))
print('Best alpha:', regressor.alpha_)

Lasso validation set- R2： 0.08137623260731919
Best alpha: 1.7620147449330423


In [36]:
# 4.使用最优的alpha值运用Lasso模型
from sklearn.linear_model import Lasso

# Create a Lasso object with the best alpha value
# regressor = Lasso(alpha=regressor.alpha_)
regressor = Lasso(alpha=1)
regressor.fit(X_train_all, y_train_all)

# Predict on the test set
model_5_y_hat_group1 = regressor.predict(X_test_1)
model_5_y_hat_group2 = regressor.predict(X_test_2)
model_5_y_hat_group3 = regressor.predict(X_test_3)

输出结果

In [37]:
# 个体level
Model_5_R2=[]
Model_5_R2.append(r2_score(y_train_all,regressor.predict(X_train_all)))
Model_5_R2.append(r2_score(test_df_group1[target],regressor.predict(X_test_1)))
Model_5_R2.append(r2_score(test_df_group2[target],regressor.predict(X_test_2)))
Model_5_R2.append(r2_score(test_df_group3[target],regressor.predict(X_test_3)))

print("model_5_R2:",Model_5_R2)

model_5_R2: [0.08391654411493898, 0.04721802785168083, -177.1737411076267, -4887733015.884496]


In [38]:
# group ATE level
model_5_ATE=[]
model_5_group_1_ATE= np.mean(model_5_y_hat_group1)- np.mean(test_df_control["session_time"])
model_5_group_2_ATE= np.mean(model_5_y_hat_group2)- np.mean(test_df_control["session_time"])
model_5_group_3_ATE= np.mean(model_5_y_hat_group3)- np.mean(test_df_control["session_time"])
model_5_ATE.append(model_5_group_1_ATE)
model_5_ATE.append(model_5_group_2_ATE)
model_5_ATE.append(model_5_group_3_ATE)

print("model_5_ATE:",model_5_ATE)

model_5_ATE: [-258.23525901468884, -73385.84554041349, 100375391.22453539]


# Model 6: f1+ f2 + f3 + f4 + f5 + f6 + c1 + c2 + c3 + c4

In [19]:
import statsmodels.formula.api as smf

#Model 4: 
# 选择特征
features = ['stay_time_s','rate_c1', 'rate_c2', 'rate_c3', 'rate_c4','rate_f1', 'rate_f2','rate_f3','rate_f4','rate_f5','rate_f6' ]
# 选择目标
target = 'session_time'

# 指定线性回归模型公式
formula = "session_time ~ stay_time_s + rate_f1 + rate_f2 + rate_f3 + rate_f4 + rate_f5 + rate_f6 + rate_c1 + rate_c2 + rate_c3 + rate_c4"    
# 在训练集上拟合线性回归模型
model_6 = smf.ols(formula=formula, data=train_df).fit()

# results
print(model_6.summary())

# 在测试集上进行预测
model_6_y_hat_group1 = model_6.predict(test_df_group1[features])
model_6_y_hat_group2 = model_6.predict(test_df_group2[features])
model_6_y_hat_group3 = model_6.predict(test_df_group3[features])

                            OLS Regression Results                            
Dep. Variable:           session_time   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                 2.934e+04
Date:                Thu, 30 Mar 2023   Prob (F-statistic):               0.00
Time:                        15:38:00   Log-Likelihood:            -3.2963e+07
No. Observations:             3263645   AIC:                         6.593e+07
Df Residuals:                 3263634   BIC:                         6.593e+07
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    2403.6130     11.390    211.027      

In [20]:
# 个体level
Model_6_R2=[]
Model_6_R2.append(r2_score(train_df[target],model_6.predict(train_df[features])))
Model_6_R2.append(r2_score(test_df_group1[target],model_6.predict(test_df_group1[features])))
Model_6_R2.append(r2_score(test_df_group2[target],model_6.predict(test_df_group2[features])))
Model_6_R2.append(r2_score(test_df_group3[target],model_6.predict(test_df_group3[features])))

print("model_6_R2:",Model_6_R2)

model_6_R2: [0.08247450970753167, 0.0481327020905189, 0.039695738251304236, 0.06790342629582424]


In [21]:
model_6_ATE=[]
model_6_group_1_ATE= np.mean(model_6_y_hat_group1)- np.mean(test_df_control["session_time"])
model_6_group_2_ATE= np.mean(model_6_y_hat_group2)- np.mean(test_df_control["session_time"])
model_6_group_3_ATE= np.mean(model_6_y_hat_group3)- np.mean(test_df_control["session_time"])
model_6_ATE.append(model_6_group_1_ATE)
model_6_ATE.append(model_6_group_2_ATE)
model_6_ATE.append(model_6_group_3_ATE)

print("model_6_ATE:",model_6_ATE)

model_6_ATE: [-163.23112766718623, 71.29443557526929, -1.6379926334766424]


# Model 7: 加上交互项：X*T 

In [22]:
import statsmodels.formula.api as smf

#Model 4: 
# 选择特征
features = ['stay_time_s','rate_f1c1', 'rate_f1c2', 'rate_f1c3', 'rate_f1c4','rate_f2c1', 'rate_f2c2', 'rate_f2c3', 'rate_f2c4','rate_f3c1', 'rate_f3c2', 'rate_f3c3', 'rate_f3c4','rate_f4c1', 'rate_f4c2', 'rate_f4c3', 'rate_f4c4','rate_f5c1', 'rate_f5c2', 'rate_f5c3', 'rate_f5c4','rate_f6c1', 'rate_f6c2', 'rate_f6c3', 'rate_f6c4']

# 选择目标
target = 'session_time'

# 指定线性回归模型公式
formula = "session_time ~ stay_time_s*(rate_f1c1 + rate_f1c2 + rate_f1c3 + rate_f1c4 + rate_f2c1 + rate_f2c2+ rate_f2c3+ rate_f2c4 + rate_f3c1 + rate_f3c2 + rate_f3c3 + rate_f3c4 + rate_f4c1 + rate_f4c2 + rate_f4c3 + rate_f4c4 + rate_f5c1 + rate_f5c2 + rate_f5c3 + rate_f5c4 + rate_f6c1 + rate_f6c2 + rate_f6c3 + rate_f6c4)"
   
# 在训练集上拟合线性回归模型
model_7 = smf.ols(formula=formula, data=train_df).fit()

# results
print(model_7.summary())

# 在测试集上进行预测
model_7_y_hat_group1 = model_7.predict(test_df_group1[features])
model_7_y_hat_group2 = model_7.predict(test_df_group2[features])
model_7_y_hat_group3 = model_7.predict(test_df_group3[features])

                            OLS Regression Results                            
Dep. Variable:           session_time   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.086
Method:                 Least Squares   F-statistic:                     9913.
Date:                Thu, 30 Mar 2023   Prob (F-statistic):               0.00
Time:                        15:38:35   Log-Likelihood:            -3.2957e+07
No. Observations:             3263645   AIC:                         6.591e+07
Df Residuals:                 3263613   BIC:                         6.591e+07
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              2355.45

In [23]:
# 个体level
Model_7_R2=[]
Model_7_R2.append(r2_score(train_df[target],model_7.predict(train_df[features])))
Model_7_R2.append(r2_score(test_df_group1[target],model_7.predict(test_df_group1[features])))
Model_7_R2.append(r2_score(test_df_group2[target],model_7.predict(test_df_group2[features])))
Model_7_R2.append(r2_score(test_df_group3[target],model_7.predict(test_df_group3[features])))

print("model_7_R2:",Model_7_R2)

model_7_R2: [0.08605747954653631, -0.1593677613508908, -1.3028839823183138, -1.0657647069075002]


In [24]:
model_7_ATE=[]
model_7_group_1_ATE= np.mean(model_7_y_hat_group1)- np.mean(test_df_control["session_time"])
model_7_group_2_ATE= np.mean(model_7_y_hat_group2)- np.mean(test_df_control["session_time"])
model_7_group_3_ATE= np.mean(model_7_y_hat_group3)- np.mean(test_df_control["session_time"])
model_7_ATE.append(model_7_group_1_ATE)
model_7_ATE.append(model_7_group_2_ATE)
model_7_ATE.append(model_7_group_3_ATE)

print("model_7_ATE:",model_7_ATE)

model_7_ATE: [1394.821474216907, 3997.5191277827266, 4511.786440241345]


# Model 8: X*(f1 + f2 + f3 + f4 + f5 + f6)

In [25]:
import statsmodels.formula.api as smf

#Model 8: 
# 选择特征
features = ['stay_time_s','rate_f1', 'rate_f2', 'rate_f3', 'rate_f4','rate_f5', 'rate_f6']
# 选择目标
target = 'session_time'

# 指定线性回归模型公式
formula = "session_time ~ stay_time_s *(rate_f1 + rate_f2 + rate_f3 + rate_f4 + rate_f5 + rate_f6)"    
# 在训练集上拟合线性回归模型
model_8 = smf.ols(formula=formula, data=train_df).fit()

# results
print(model_8.summary())

# 在测试集上进行预测
model_8_y_hat_group1 = model_8.predict(test_df_group1[features])
model_8_y_hat_group2 = model_8.predict(test_df_group2[features])
model_8_y_hat_group3 = model_8.predict(test_df_group3[features])

                            OLS Regression Results                            
Dep. Variable:           session_time   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                 2.261e+04
Date:                Thu, 30 Mar 2023   Prob (F-statistic):               0.00
Time:                        15:38:53   Log-Likelihood:            -3.2963e+07
No. Observations:             3263645   AIC:                         6.593e+07
Df Residuals:                 3263631   BIC:                         6.593e+07
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept            2383.6230    

In [26]:
# 个体level
Model_8_R2=[]
Model_8_R2.append(r2_score(train_df[target],model_8.predict(train_df[features])))
Model_8_R2.append(r2_score(test_df_group1[target],model_8.predict(test_df_group1[features])))
Model_8_R2.append(r2_score(test_df_group2[target],model_8.predict(test_df_group2[features])))
Model_8_R2.append(r2_score(test_df_group3[target],model_8.predict(test_df_group3[features])))

print("model_8_R2:",Model_8_R2)

model_8_R2: [0.08260855748480844, 0.04770997482145756, 0.03450916762339307, 0.06734130351163581]


In [27]:
model_8_ATE=[]
model_8_group_1_ATE= np.mean(model_8_y_hat_group1)- np.mean(test_df_control["session_time"])
model_8_group_2_ATE= np.mean(model_8_y_hat_group2)- np.mean(test_df_control["session_time"])
model_8_group_3_ATE= np.mean(model_8_y_hat_group3)- np.mean(test_df_control["session_time"])
model_8_ATE.append(model_8_group_1_ATE)
model_8_ATE.append(model_8_group_2_ATE)
model_8_ATE.append(model_8_group_3_ATE)

print("model_8_ATE:",model_8_ATE)

model_8_ATE: [40.46673060433841, 102.1414254467827, 34.30920350147608]


# Model 9: X*(c1 + c2 + c3 +c4)

In [28]:
import statsmodels.formula.api as smf

#Model 4: 
# 选择特征
features = ['stay_time_s','rate_c1', 'rate_c2', 'rate_c3', 'rate_c4']
# 选择目标
target = 'session_time'

# 指定线性回归模型公式
formula = "session_time ~ stay_time_s*(rate_c1 + rate_c2 + rate_c3 + rate_c4)"    
# 在训练集上拟合线性回归模型
model_9 = smf.ols(formula=formula, data=train_df).fit()

# results
print(model_9.summary())

# 在测试集上进行预测
model_9_y_hat_group1 = model_9.predict(test_df_group1[features])
model_9_y_hat_group2 = model_9.predict(test_df_group2[features])
model_9_y_hat_group3 = model_9.predict(test_df_group3[features])

                            OLS Regression Results                            
Dep. Variable:           session_time   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                 3.255e+04
Date:                Thu, 30 Mar 2023   Prob (F-statistic):               0.00
Time:                        15:39:01   Log-Likelihood:            -3.2964e+07
No. Observations:             3263645   AIC:                         6.593e+07
Df Residuals:                 3263635   BIC:                         6.593e+07
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept            2380.5688    

In [29]:
# 个体level
Model_9_R2=[]
Model_9_R2.append(r2_score(train_df[target],model_9.predict(train_df[features])))
Model_9_R2.append(r2_score(test_df_group1[target],model_9.predict(test_df_group1[features])))
Model_9_R2.append(r2_score(test_df_group2[target],model_9.predict(test_df_group2[features])))
Model_9_R2.append(r2_score(test_df_group3[target],model_9.predict(test_df_group3[features])))

print("model_9_R2:",Model_9_R2)

model_9_R2: [0.08235905733001392, 0.05090029448391131, 0.041702666207442673, 0.0679380716951643]


In [30]:
model_9_ATE=[]
model_9_group_1_ATE= np.mean(model_9_y_hat_group1)- np.mean(test_df_control["session_time"])
model_9_group_2_ATE= np.mean(model_9_y_hat_group2)- np.mean(test_df_control["session_time"])
model_9_group_3_ATE= np.mean(model_9_y_hat_group3)- np.mean(test_df_control["session_time"])
model_9_ATE.append(model_9_group_1_ATE)
model_9_ATE.append(model_9_group_2_ATE)
model_9_ATE.append(model_9_group_3_ATE)

print("model_9_ATE:",model_9_ATE)

model_9_ATE: [-177.49826208401146, -49.221525130517875, -16.566812605032283]


# 5. Compare all Models

# ---------------------------- first 5 models------------------------------------------

In [39]:
all_model_indi_R2= pd.DataFrame(dict(
    model_1_R2=Model_1_R2,
    model_2_R2=Model_2_R2,
    model_3_R2=Model_3_R2,
    model_4_R2=Model_4_R2,
    model_5_R2=Model_5_R2,
))
all_model_indi_R2

,model_1_R2,model_2_R2,model_3_R2,model_4_R2,model_5_R2
0,2.581286e-03,0.084092,0.082129,0.081971,8.391654e-02
1,-8.630614e+20,-0.006344,0.048392,0.048024,4.721803e-02
2,-2.493703e+18,-0.519334,0.039315,0.040532,-1.771737e+02
3,-1.512091e+18,-0.487651,0.067919,0.067875,-4.887733e+09


In [40]:
all_ATE= pd.DataFrame(dict(
    real_ATE=real_ATE,
    model_1_ATE=model_1_ATE,
    model_2_ATE=model_2_ATE,
    model_3_ATE=model_3_ATE,
    model_4_ATE=model_4_ATE,
    model_5_ATE=model_5_ATE,
))
all_ATE

,real_ATE,model_1_ATE,model_2_ATE,model_3_ATE,model_4_ATE,model_5_ATE
0,-38.127994,1.779121e+14,1380.888987,40.387572,-178.023975,-2.582353e+02
1,-94.315717,8.695137e+12,4025.931762,103.216154,-49.743635,-7.338585e+04
2,24.172070,-7.430652e+12,4528.275101,34.670765,-16.787217,1.003754e+08


In [26]:
# 给定两列值，Y和 Y_hat
# 计算两列的R square
def r_square(y, y_hat):
    # 计算平均值
    y_mean = np.mean(y)
    # 计算总的方差
    total_variance = np.sum((y - y_mean) ** 2)
    # 计算残差
    residual = np.sum((y - y_hat) ** 2)
    # 计算R square
    r_square = 1 - residual / total_variance
    return r_square

# 给定两列值，Y和 Y_hat
# 计算两列的MAPE
def mape(y, y_hat):
    # 计算MAPE
    mape = np.mean(np.abs((y - y_hat) / y))
    return mape

In [32]:
mape(pd.DataFrame(real_ATE),pd.DataFrame(model_4_ATE))

0    1.821367
dtype: float64

In [42]:
all_model_ATE_R2=[]
all_model_ATE_R2.append(r_square(all_ATE["real_ATE"],all_ATE["model_1_ATE"]))
all_model_ATE_R2.append(r_square(all_ATE["real_ATE"],all_ATE["model_2_ATE"]))
all_model_ATE_R2.append(r_square(all_ATE["real_ATE"],all_ATE["model_3_ATE"]))
all_model_ATE_R2.append(r_square(all_ATE["real_ATE"],all_ATE["model_4_ATE"]))
all_model_ATE_R2.append(r_square(all_ATE["real_ATE"],all_ATE["model_5_ATE"]))


In [43]:
all_model_ATE_MAPE=[]
all_model_ATE_MAPE.append(mape(all_ATE["real_ATE"],all_ATE["model_1_ATE"]))
all_model_ATE_MAPE.append(mape(all_ATE["real_ATE"],all_ATE["model_2_ATE"]))
all_model_ATE_MAPE.append(mape(all_ATE["real_ATE"],all_ATE["model_3_ATE"]))
all_model_ATE_MAPE.append(mape(all_ATE["real_ATE"],all_ATE["model_4_ATE"]))
all_model_ATE_MAPE.append(mape(all_ATE["real_ATE"],all_ATE["model_5_ATE"]))


In [44]:
all_model_ATE_metrics=pd.DataFrame(dict(
    all_model_ATE_R2=all_model_ATE_R2,
    all_model_ATE_MAPE=all_model_ATE_MAPE
))
all_model_ATE_metrics

,all_model_ATE_R2,all_model_ATE_MAPE
0,-4.523762e+24,1.688593e+12
1,-5.589311e+03,8.907930e+01
2,-5.446680e+00,1.529321e+00
3,-2.307079e+00,1.945396e+00
4,-1.434010e+12,1.384439e+06


In [45]:
all_model_indi_R2.to_excel("all_model_indi_R2.xls")
all_ATE.to_excel("all_ATE.xls")
all_model_ATE_metrics.to_excel("all_model_ATE_metrics.xls")

# -------------------------------rest models----------------------------

In [34]:
all_model_indi_R2= pd.DataFrame(dict(
    model_6_R2=Model_6_R2,
    model_7_R2=Model_7_R2,
    model_8_R2=Model_8_R2,
    model_9_R2=Model_9_R2
))
all_model_indi_R2

,model_6_R2,model_7_R2,model_8_R2,model_9_R2
0,0.082475,0.086057,0.082609,0.082359
1,0.048133,-0.159368,0.047710,0.050900
2,0.039696,-1.302884,0.034509,0.041703
3,0.067903,-1.065765,0.067341,0.067938


In [35]:
all_ATE= pd.DataFrame(dict(
    real_ATE=real_ATE,
    model_6_ATE=model_6_ATE,
    model_7_ATE=model_7_ATE,
    model_8_ATE=model_8_ATE,
    model_9_ATE=model_9_ATE
))
all_ATE

,real_ATE,model_6_ATE,model_7_ATE,model_8_ATE,model_9_ATE
0,-38.127994,-163.231128,1394.821474,40.466731,-177.498262
1,-94.315717,71.294436,3997.519128,102.141425,-49.221525
2,24.172070,-1.637993,4511.786440,34.309204,-16.566813


In [38]:
all_model_ATE_R2=[]
all_model_ATE_R2.append(r_square(all_ATE["real_ATE"],all_ATE["model_6_ATE"]))
all_model_ATE_R2.append(r_square(all_ATE["real_ATE"],all_ATE["model_7_ATE"]))
all_model_ATE_R2.append(r_square(all_ATE["real_ATE"],all_ATE["model_8_ATE"]))
all_model_ATE_R2.append(r_square(all_ATE["real_ATE"],all_ATE["model_9_ATE"]))

In [39]:
all_model_ATE_MAPE=[]
all_model_ATE_MAPE.append(mape(all_ATE["real_ATE"],all_ATE["model_6_ATE"]))
all_model_ATE_MAPE.append(mape(all_ATE["real_ATE"],all_ATE["model_7_ATE"]))
all_model_ATE_MAPE.append(mape(all_ATE["real_ATE"],all_ATE["model_8_ATE"]))
all_model_ATE_MAPE.append(mape(all_ATE["real_ATE"],all_ATE["model_9_ATE"]))

In [40]:
all_model_ATE_metrics=pd.DataFrame(dict(
    all_model_ATE_R2=all_model_ATE_R2,
    all_model_ATE_MAPE=all_model_ATE_MAPE
))
all_model_ATE_metrics

,all_model_ATE_R2,all_model_ATE_MAPE
0,-5.226056,2.034937
1,-5540.654940,88.873309
2,-5.387121,1.521229
3,-2.290283,1.939605


In [41]:
all_model_indi_R2.to_excel("all_model_indi_R2_1.xls")
all_ATE.to_excel("all_ATE_1.xls")
all_model_ATE_metrics.to_excel("all_model_ATE_metrics_1.xls")